In [25]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from simulation_functions import *
from tqdm import tqdm
from scipy.integrate import solve_ivp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
ks = load_3d_matrix_from_csv("many_ks.csv")
Ks = load_3d_matrix_from_csv("many_KKs.csv")

In [27]:
K = Ks[-5,:,:]
k = ks[-5,:,:]
g = K.shape[1]
lam = 5
lam = np.zeros(g)+lam
t_f = 200
n0 = np.zeros(g) + 0.5
nt, t = sim_dyn(n0,t_f,k,K,lam)
n_f = nt[:,-1]
J = calc_J(k,K,n_f,lam)
w, v = np.linalg.eig(J)

t_f = 5
m = 20
g = len(lam)
lam_c_0 = np.random.rand(g)-0.5 
s_hat_0 = np.random.rand(g)-0.5 
s_hat_0 = s_hat_0/np.linalg.norm(s_hat_0)
temperature = 1.0
cooling_rate = 0.96
num_iterations = 1000
best_s_hat, best_lam_c, costs = simulated_annealing(k,K,n_f,lam,lam_c_0,s_hat_0,m,t_f, num_iterations, temperature, cooling_rate,g)

100%|██████████████████████████████████████| 1000/1000 [04:29<00:00,  3.71it/s]


In [30]:
np.array([5]).shape

(1,)